In [1]:
from openagentkit.modules.openai import OpenAIExecutor
from openagentkit.core.utils.tool_wrapper import tool
from pydantic import BaseModel
from typing import Annotated
import openai
import os

In [2]:
# Define a tool
@tool(
    description="Get the weather of a city", # Define the tool description
)
def get_weather(city: Annotated[str, "The city to get the weather of"]): # Each argument must be of type Annotated
    """Get the weather of a city"""

    # Actual implementation here...
    # ...

    return f"Weather in {city}: sunny, 20°C, feels like 22°C, humidity: 50%"

In [ ]:
# Initialize OpenAI client
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Initialize LLM service
executor = OpenAIExecutor(
    client=client,
    model="gpt-4o-mini",
    system_message="""
    You are a helpful assistant that can answer questions and help with tasks.
    You are also able to use tools to get information.
    """,
    tools=[get_weather],
    temperature=0.5,
    max_tokens=100,
    top_p=1.0,
)

generator = executor.execute(
    messages=[
        {"role": "user", "content": "What's the weather like in New York?"}
    ]
)

for response in generator:
    print(response)

2025-04-17 22:33:08.350 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content=None tool_calls=[{'id': 'call_4VGCifNwxF1mzVU5UMsaKJKr', 'type': 'function', 'function': {'arguments': '{"city":"New York"}', 'name': 'get_weather'}}] tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=112, completion_tokens=16, total_tokens=128, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))
2025-04-17 22:33:08.352 | INFO     | openagentkit.core.handlers.tool_handler:handle_tool_request:177 - Tool Result: Weather in New York: sunny, 20°C, feels like 22°C, humidity: 50%
2025-04-17 22:33:09.619 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content='The weather in New York is currently sunny, w

The weather in New York is currently sunny, with a temperature of 20°C (feels like 22°C) and a humidity level of 50%.


In [4]:
executor.clear_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-04-17 22:33:06\n\n        \n        '}]

In [5]:
while True:
    user_input = input("User: ")
    print(f"User: {user_input}")
    
    if user_input.lower() == "exit":
        break

    generator = executor.execute(
        messages=[
            {"role": "user", "content": user_input}
        ]
    )

    for response in generator:
        print(f"AI: {response.content}")
    if response.tool_results:
            print(f"Tool response: {response.tool_response}")

User: Hello!


2025-04-17 22:33:13.470 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content='Hello! How can I assist you today?' tool_calls=None tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=106, completion_tokens=11, total_tokens=117, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))


AI: Hello! How can I assist you today?
User: I'd like to know about the weather


2025-04-17 22:33:26.463 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content='Sure! Which city would you like to know the weather for?' tool_calls=None tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=143, completion_tokens=15, total_tokens=158, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))


AI: Sure! Which city would you like to know the weather for?
User: Hmm I'm in Hanoi


2025-04-17 22:33:40.070 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content=None tool_calls=[{'id': 'call_AtjqOz7ql30K5EvzGHTjgVxg', 'type': 'function', 'function': {'arguments': '{"city":"Hanoi"}', 'name': 'get_weather'}}] tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=185, completion_tokens=16, total_tokens=201, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))
2025-04-17 22:33:40.071 | INFO     | openagentkit.core.handlers.tool_handler:handle_tool_request:177 - Tool Result: Weather in Hanoi: sunny, 20°C, feels like 22°C, humidity: 50%
2025-04-17 22:33:42.186 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content='The weather in Hanoi is currently sunny, with a tem

AI: The weather in Hanoi is currently sunny, with a temperature of 20°C. It feels like 22°C, and the humidity is at 50%. If you need more information or have other questions, feel free to ask!
User: Nice one, have a great day!


2025-04-17 22:33:51.795 | INFO     | openagentkit.modules.openai.openai_executor:execute:164 - Response Received: role='assistant' content='Thank you! You have a great day too! If you have any more questions in the future, feel free to ask.' tool_calls=None tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=346, completion_tokens=27, total_tokens=373, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0))


AI: Thank you! You have a great day too! If you have any more questions in the future, feel free to ask.
User: exit
